In [1]:
import requests
from os import listdir
from os.path import isfile, join
import json
import time

In [2]:
api_url = "http://localhost:8080/api/v1/"
partial_seed_url = api_url + "seed/partial"
full_seed_url = api_url + "seed/partial"
transactions_url = api_url + "seed/transactions"
transaction_url = api_url + "seed/transaction"
jobs_url = api_url + "jobs"
headers = {"Authorization": "token test-key", "Content-Type":"application/json"}

In [3]:
valid_person_data_path = "person-data/valid/"
invalid_person_data_path = "person-data/invalid"
valid_person_data = [valid_person_data_path+f for f in listdir(valid_person_data_path) if isfile(join(valid_person_data_path, f))]
invalid_person_data = [invalid_person_data_path+f for f in listdir(invalid_person_data_path) if isfile(join(invalid_person_data_path, f))]

In [4]:
invalid_person_data

['person-data/invalidnatural-person2.json',
 'person-data/invalidnatural-person1.json']

In [5]:
def load_json(filename):
    json_file = open(filename)
    return json_file.read()

In [6]:
load_json("person-data/valid/natural-person1.json")

'{\n  "metaData": {\n    "personType": "NATUERLICHE_PERSON",\n    "eventType": "INSERT"\n  },\n  "natuerlichePerson": {\n    "name": "Smith",\n    "vorname": "John",\n    "jahrgang": "1970"\n  }\n}'

In [7]:
def send_data(url, headers, data_list):
    transaction_ids = []
    for data in data_list:
        response = requests.post(partial_seed_url, headers=headers, data=data)
        assert response.status_code == 200
            
        transaction_ids.append(response.json()["transactionId"])
    return transaction_ids        

In [8]:
def run_tests():
    return {
        "ids_valid_person_data":send_data(partial_seed_url, headers, list(map(load_json, valid_person_data))),
        "ids_invalid_person_data":send_data(partial_seed_url, headers, list(map(load_json, invalid_person_data))),
    }
    

In [10]:
def get_transaction(transactionId):
    json_response = requests.get(transaction_url+"/"+transactionId, headers=headers).json()
    return json_response

In [11]:
get_transaction("218bd690-cabb-4329-b720-2b14f8c7d957")

{'transactionId': '218bd690-cabb-4329-b720-2b14f8c7d957',
 'state': 'FAILED',
 'jobId': None}

In [12]:
def wait_for_completion(transaction_id):
    transaction = get_transaction("218bd690-cabb-4329-b720-2b14f8c7d957")
    processing = True
    trials = 0
    while(processing and trials < 10):
        trials = trials + 1
        if(transaction['state'] in ["PROCESSED", "FAILED"]):
            return transaction['state']
        else:
            sleep(1)
            transaction = get_transaction("218bd690-cabb-4329-b720-2b14f8c7d957")
    return "STALE"
        

In [13]:
list(zip(valid_person_data,map(load_json, valid_person_data)))

[('person-data/valid/natural-person2.json',
  '{\n  "metaData": {\n    "personType": "NATUERLICHE_PERSON",\n    "eventType": "INSERT"\n  },\n  "natuerlichePerson": {\n    "name": "Müller",\n    "vorname": "Hans",\n    "jahrgang": "1970"\n  }\n}'),
 ('person-data/valid/natural-person3.json',
  '{\n  "metaData": {\n    "personType": "JURISITSCHE_PERSON",\n    "eventType": "INSERT"\n  },\n  "natuerlichePerson": {\n    "name": "Müller",\n    "vorname": "Hans",\n    "jahrgang": "1970"\n  }\n}'),
 ('person-data/valid/natural-person1.json',
  '{\n  "metaData": {\n    "personType": "NATUERLICHE_PERSON",\n    "eventType": "INSERT"\n  },\n  "natuerlichePerson": {\n    "name": "Smith",\n    "vorname": "John",\n    "jahrgang": "1970"\n  }\n}')]

In [14]:
get_transaction("6ac23b82-0ee8-4054-bf64-6596e5c01180")

{'transactionId': '6ac23b82-0ee8-4054-bf64-6596e5c01180',
 'state': 'FAILED',
 'jobId': None}